# **1. Installation de Yolo Ultralytics**

In [ ]:
# !pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


# **2. Configuration et téléchargement de la base de données**

In [1]:
# Configuration
epochs = 50 #@param [5,10,20,50,100,200] {type:"raw"}
batch_size = 24 #@param [8,16,32,64] {type:"raw"}
imgsz = 640 #@param [640] {type:"raw"}
classes = ['smoke', 'fire']

# Percentage of data to keep per split (train/valid/test)
keep_pct = 20 #@param [10,20,30,50,100]
random_seed = 42

yolo_version = "yolo11n" #@param ["yolov11n", "yolov11s","yolov11m","yolov11l","yolov11x"] {type:"string"}
import os, random, shutil, glob


In [8]:
# Download dataset
# !wget https://nextcloud.ig.umons.ac.be/s/2RwG4yCcQ8RQASE/download/d_fire.zip -O d_fire.zip
# !unzip -q d_fire.zip -d /content/fire_detection_dataset/

# Create a reduced dataset with only `keep_pct` of images per split
import os, random, shutil, glob

random.seed(random_seed)
src_root = "./content/fire_detection_dataset/D-Fire"
dst_root = "./content/fire_detection_dataset/D-Fire_reduced"


os.makedirs(dst_root, exist_ok=True)

splits = ["train", "valid", "test"]
img_exts = {".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"}

for split in splits:
    src_img_dir = os.path.join(src_root, "images", split)
    src_lbl_dir = os.path.join(src_root, "labels", split)
    dst_img_dir = os.path.join(dst_root, "images", split)
    dst_lbl_dir = os.path.join(dst_root, "labels", split)
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    image_files = [f for f in glob.glob(os.path.join(src_img_dir, "*")) if os.path.splitext(f)[1] in img_exts]
    if not image_files:
        continue
    k = max(1, int(len(image_files) * keep_pct / 100.0))
    sampled = sorted(random.sample(image_files, k))
    for img_path in sampled:
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(src_lbl_dir, base + ".txt")
        shutil.copy2(img_path, os.path.join(dst_img_dir, os.path.basename(img_path)))
        if os.path.isfile(lbl_path):
            shutil.copy2(lbl_path, os.path.join(dst_lbl_dir, os.path.basename(lbl_path)))

print(f"Created reduced dataset at {dst_root}")

# Create dataset config
import yaml
config = {
    "path": dst_root,
    "train": "images/train",
    "val": "images/valid",
    "test": "images/test",
    "names": classes
}
with open('dataset.yaml', 'w') as f:
    yaml.dump(config, f)


Created reduced dataset at ./content/fire_detection_dataset/D-Fire_reduced


# **3. Entrainement et évaluation du modèle**

In [2]:
from ultralytics import YOLO

# Load YOLOv11 model
model = YOLO(f"{yolo_version}.pt")

# Train the model with early stopping and checkpointing
results = model.train(
    data='dataset.yaml',
    epochs=epochs,
    batch=batch_size,
    imgsz=imgsz,
    device=0,
    save=True,   # Save the best model checkpoint
    save_period=1  # Save checkpoint every epoch
)

# Evaluate the model
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.14.0 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3770MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opse

# **4. Evaluation du modèle avec une image de test**

In [4]:
from ultralytics import YOLO

# Reload the YOLO model from the saved checkpoint
# model = YOLO('./runs/detect/train3/weights/best.pt')  # Adjust the path to your saved model checkpoint

import matplotlib.pyplot as plt
import random

# Get 25 random images from the reduced dataset
all_images = glob.glob('./content/fire_detection_dataset/D-Fire_reduced/images/train/*') + \
             glob.glob('./content/fire_detection_dataset/D-Fire_reduced/images/valid/*')

random_images = random.sample(all_images, min(25, len(all_images)))

# Set up the plot grid
fig, axes = plt.subplots(5, 5, figsize=(15, 15))
axes = axes.flatten()

# Perform inference and plot each image
for ax, img_path in zip(axes, random_images):
    results = model(img_path)
    result = results[0]
    result_img = result.plot()  # Get the image with predictions drawn on it
    ax.imshow(result_img)
    ax.axis('off')
    ax.set_title(f"{os.path.basename(img_path)}")

# Hide any unused subplots
for ax in axes[len(random_images):]:
    ax.axis('off')

plt.tight_layout()
plt.show()



image 1/1 /run/media/cyril/Windows/Users/cyril/Desktop/jjj/info/Ma2/defi IA/tp/intro/projet/content/fire_detection_dataset/D-Fire_reduced/images/train/WEB06250.jpg: 352x640 1 smoke, 1 fire, 16.1ms
Speed: 1.1ms preprocess, 16.1ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /run/media/cyril/Windows/Users/cyril/Desktop/jjj/info/Ma2/defi IA/tp/intro/projet/content/fire_detection_dataset/D-Fire_reduced/images/train/WEB04467.jpg: 448x640 2 smokes, 1 fire, 20.1ms
Speed: 1.3ms preprocess, 20.1ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /run/media/cyril/Windows/Users/cyril/Desktop/jjj/info/Ma2/defi IA/tp/intro/projet/content/fire_detection_dataset/D-Fire_reduced/images/train/WEB03070.jpg: 384x640 (no detections), 16.4ms
Speed: 0.7ms preprocess, 16.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /run/media/cyril/Windows/Users/cyril/Desktop/jjj/info/Ma2/defi IA/tp/intro/projet/content/fire_detectio

<Figure size 1500x1500 with 25 Axes>

résultat yolo_version
tous entrainé sur le petit dataset


pour yolo11s a obtenu : à rerun
    


pour yolo11n a obtenu
batchsize 24 
mAP50: 0.960
mAP50-95: 0.778
prend 208MiB de ram et tourne à 140FPS



yolo11m: 
bs 6 car pas assez de ram
map50 = 0,96
map50-95 = 0,775
batch size pas assez grand mais pas assez de ram
utilise que 364 MiB de ram pour quand l'utilise mais sur 3050 attend que
et tourne à 60 PFS


plus lourd modèle semble pas interessant car si c'est pour faire du edge AI alors sera beaucoup trop lourd